# Supervised Learning – RUL Prediction
This notebook trains LSTM and XGBoost models to predict Remaining Useful Life (RUL) of EV batteries from CALCE dataset.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from src.models.lstm import build_lstm
import tensorflow as tf

## Load Data

In [ ]:
df = pd.read_csv('../data/processed/cell01.csv')
X = df[['voltage','current','temperature']].values
RUL = len(df) - df['cycle'].values
X_train, X_test, y_train, y_test = train_test_split(X, RUL, test_size=0.2, shuffle=False)

## XGBoost Baseline

In [ ]:
xgb = XGBRegressor(n_estimators=300, max_depth=6, learning_rate=0.05)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_test)
print("XGB RMSE:", mean_squared_error(y_test, preds, squared=False))
print("XGB MAE:", mean_absolute_error(y_test, preds))

## LSTM Model

In [ ]:
X_train_lstm = np.expand_dims(X_train, axis=1)
X_test_lstm = np.expand_dims(X_test, axis=1)
lstm = build_lstm((1, X_train.shape[1]))
history = lstm.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_split=0.2)
preds_lstm = lstm.predict(X_test_lstm).flatten()
print("LSTM RMSE:", mean_squared_error(y_test, preds_lstm, squared=False))
print("LSTM R²:", r2_score(y_test, preds_lstm))